In [1]:
from IPython.core.display import HTML
with open('../../style.css') as file:
    css = file.read()
HTML(css)

# Embedded Actions in <span style="font-variant:small-caps;">Antlr</span> Grammars

The *pure grammar* is stored in the file `Grammar.g4`.  On *Unix-like* operating
systems, the following command displays the file `Program.g4`.  

In [2]:
!cat -n Program.g4

'cat' is not recognized as an internal or external command,
operable program or batch file.


On a *Windows* operating system the cell given below prints the contend of
the file `Program.g4`.

In [3]:
!type Program.g4

grammar Program;

start: statement+ ; 

statement
    : IDENTIFIER ':=' expr ';' 
    | expr ';'                 
    ;

expr: expr '+' product 
    | expr '-' product 
    | product
    ;

product 
    : product '*' factor 
    | product '/' factor 
    | factor
    ;

factor
    : 'sqrt' '(' expr ')'
    | '(' expr ')'        
    | FLOAT               
    | IDENTIFIER          
    ;

IDENTIFIER: [a-zA-Z][a-zA-Z0-9_]*;
FLOAT     : '0'([.][0-9]+)?
          | [1-9][0-9]*([.][0-9]+)?;
WS        : [ \t\n\r] -> skip; 


The grammar shown above has no *semantic actions* (with the exception of the `skip` action). 
We extend this grammar now with *semantic actions* so that we can actually compute something.
This grammar is stored in the file `Calculator.g4`.  It describes a language for a 
*symbolic calculator*: This calculator is able to evaluate arithmetic expressions and, furthermore, 
where we can store the results of our computations in variables.  

In [ ]:
!cat -n Calculator.g4

In [4]:
!type Calculator.g4

grammar Calculator;

@header {
import math
}

start: statement+ ; 

statement
    : IDENTIFIER ':=' expr ';' {self.Values[$IDENTIFIER.text] = $expr.result}
    | expr ';'                 {print($expr.result)                         }
    ;

expr returns[result]
    : e=expr '+' p=product {$result = $e.result + $p.result}
    | e=expr '-' p=product {$result = $e.result - $p.result}
    | p=product            {$result = $p.result            }
    ;

product returns[result]
    : p=product '*' f=factor {$result = $p.result * $f.result}
    | p=product '/' f=factor {$result = $p.result / $f.result}
    | f=factor               {$result = $f.result            }
    ;

factor returns[result]
    : 'sqrt' '(' expr ')' {$result = math.sqrt($expr.result)      }
    | '(' expr ')'        {$result = $expr.result                 }
    | FLOAT               {$result = float($FLOAT.text)           }
    | IDENTIFIER          {$result = self.Values[$IDENTIFIER.text]}
    ;

IDENTIFIER: [a-zA-Z][a-zA-

First, we have to generate both the scanner and the parser.  

In [5]:
!antlr4 -Dlanguage=Python3 Calculator.g4


C:\Users\Karl\Dropbox\Kurse\Formal-Languages\ANTLR4-Python\Calculator>java -Xmx500M -cp C:/Users/Karl/anaconda3/envs/fl\Library\lib\\antlr4-4.9.2_1-complete.jar; org.antlr.v4.Tool -Dlanguage=Python3 Calculator.g4  

C:\Users\Karl\Dropbox\Kurse\Formal-Languages\ANTLR4-Python\Calculator>IF 0 NEQ 0 EXIT /B 0  


We can use the system command `ls` to see which files have been generated by <span style="font-variant:small-caps;">Antlr</span>.
If you are using a windows system you have to use the command `dir` instead.

In [ ]:
!ls -l

In [ ]:
!dir

The files `CalculatorLexer.py` and `CalculatorParser.py` contain the generated scanner and parser, respectively.  We have to import these files.  Furthermore, the runtime of 
<span style="font-variant:small-caps;">Antlr</span>
needs to be imported.

In [ ]:
from CalculatorLexer  import CalculatorLexer
from CalculatorParser import CalculatorParser
import antlr4

Let us parse and evaluate the input that we read from a prompt.

In [ ]:
def main():
    parser        = CalculatorParser(None) # generate parser without lexer
    parser.Values = {}
    line          = input('> ')
    while line != '':
        input_stream  = antlr4.InputStream(line)
        lexer         = CalculatorLexer(input_stream)
        token_stream  = antlr4.CommonTokenStream(lexer)
        parser.setInputStream(token_stream)
        parser.start()
        line = input('Input an arithmetic expression: ')
    return parser.Values

In [ ]:
main()

On a *Unix-like* system, i.e. on *MacOs* or *Linux* the following cell removes the
generated files.

In [ ]:
!rm *.py *.tokens *.interp
!rm -r __pycache__/
!ls -l

On *Windows*, the following cell removes the generated files.

In [ ]:
!del *.py *.tokens *.interp
!rmdir __pycache__ /s /q
!dir